# Food Classification with CNN - Building a Restaurant Recommendation System

This assignment focuses on developing a deep learning-based food classification system using Convolutional Neural Networks (CNNs). You will build a model that can recognize different food categories and use it to return the food preferences of a user.

## Learning Objectives
- Implement CNNs for image classification
- Work with real-world food image datasets
- Build a preference-detector system

## Background: AI-Powered Food Preference Discovery

The system's core idea is simple:

1. Users upload 10 photos of dishes they enjoy
2. Your CNN classifies these images into the 91 categories
3. Based on these categories, the system returns the user's taste profile

Your task is to develop the core computer vision component that will power this detection engine.

You are given a training ("train" folder) and a test ("test" folder) dataset which have ~45k and ~22k samples respectively. For each one of the 91 classes there is a subdirectory containing the images of the respective class.

## Assignment Requirements

### Technical Requirements
- Implement your own pytorch CNN architecture for food image classification
- Use only the provided training dataset split for training
- Train the network from scratch ; No pretrained weights can be used
- Report test-accuracy after every epoch
- Report all hyperparameters of final model
- Use a fixed seed and do not use any CUDA-features that break reproducibility
- Use Pytorch 2.6

### Deliverables
1. Jupyter Notebook with CNN implementation, training code etc.
2. README file
3. Report (max 3 pages)

Submit your report, README and all code files as a single zip file named GROUP_[number]_NC2425_PA. The names and IDs of the group components must be mentioned in the README.
Do not include the dataset in your submission.

### Grading

1. Correct CNN implementation, training runs on the uni DSLab computers according to the README.MD instructions without ANY exceptions on the DSLab machines: 3pt
2. Perfect 1:1 reproducibility on DSLab machines: 1pt
3. Very clear github-repo-style README.MD with instructions for running the code: 1pt
4. Report: 1pt
5. Model test performance on test-set: interpolated from 30-80% test-accuracy: 0-3pt
6. Pick 10 random pictures of the test set to simulate a user uploading images and report which categories occur how often in these: 1pt
7. Bonus point: use an LLM (API) to generate short description / profile of preferences of the simulated user

**If there is anything unclear about this assignment please post your question in the Brightspace discussions forum or send an email**


# Loading the datasets
The dataset is already split into a train and test set in the directories "train" and "test". 

In [2]:
### HYPER-PARAMETER SET-UP ###

# Randomization
random_seed = 13

# Model training
workers = 8
batch_size = 32
num_epochs = 50
lr = 0.01
momentum = 0.8

In [3]:
### IMPORTS AND DEVICE ###

import torch
import numpy as np
import torch.nn as nn
from time import time
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.autograd import Variable
from datetime import datetime as dt
from torchvision.transforms import v2
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler


# Use the GPU to speed up the training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
### DATA LOADING ###

# Allow repeatability
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

# Define the transformer for the data
train_transform = v2.Compose([
    v2.Resize((256, 256)),
    v2.RandomApply([v2.TrivialAugmentWide()], p=0.5),
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = v2.Compose([
    v2.Resize((256, 256)),
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Collect the data and set the augmentation
train_dataset = datasets.ImageFolder(root='../train', transform=train_transform)
test_dataset = datasets.ImageFolder(root='../test', transform=test_transform) 

# Prepare the data loaders with the batches, shuffling, and workers
train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=workers)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=workers)

In [5]:
len(train_loader), len(test_loader)

(1429, 704)

# CNN Implementation

In [6]:
### ResNet ###

class Block(nn.Module):
    def __init__(self, in_channels, mid_channels, stride=1, expansion=4):
        super().__init__()
        # Convolutional block
        self.layer = nn.Sequential(nn.Conv2d(in_channels, mid_channels, 1, 1, 0),
                                   nn.SELU(),
                                   nn.BatchNorm2d(mid_channels),
                                   nn.Conv2d(mid_channels, mid_channels, 3, stride, 1),
                                   nn.SELU(),
                                   nn.Conv2d(mid_channels, mid_channels*expansion, 1, 1, 0),
                                   nn.BatchNorm2d(mid_channels*expansion))

        # In case the dimensionality of the data changes
        self.proj = False
        if in_channels != (mid_channels*expansion) or stride != 1:
            self.proj = True
            self.p_layer = nn.Conv2d(in_channels, mid_channels*4, 1, stride)

    def forward(self, x):
        res = x
        out = self.layer(x)
        # Add the residual of the original data to allow for an identity function
        if self.proj:
            res = self.p_layer(x)
        return nn.SELU()(out+res)


class ResNet(nn.Module):
    """
    Made to emulate the Residual Network.
    """
    def __init__(self, num_blocks=None):
        super().__init__()
        if num_blocks is None:
            num_blocks = [3, 4, 6, 3]
        self.layer1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
                                    nn.SELU(),
                                    nn.MaxPool2d(3, 2, 1))          # 64

        self.layer2 = self._make_layer(64, 64, num_blocks[0], 1)
        self.layer3 = self._make_layer(256, 128, num_blocks[1])     # 32
        self.layer4 = self._make_layer(512, 256, num_blocks[2])     # 16
        self.layer5 = self._make_layer(1024, 512, num_blocks[3])    # 8

        self.layer6 = nn.Sequential(nn.AvgPool2d(8),
                                    nn.Flatten(),
                                    nn.Dropout(0.25),
                                    nn.Linear(2048, 1024),
                                    nn.ReLU(),
                                    nn.Dropout(0.25),
                                    nn.Linear(1024, 91))
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        return x
    
    def _make_layer(self, in_channels, mid_channels, blocks, stride=2, expansion=4):
        layers = [Block(in_channels, mid_channels, stride, expansion)]
        for _ in range(1, blocks):
            layers.append(Block(mid_channels*expansion, mid_channels, expansion=expansion))
        return nn.Sequential(*layers)

In [7]:
x = torch.randn(batch_size, 3, 256, 256)
trial = ResNet()
print(trial(x).shape)

import gc
del x
del trial
torch.cuda.empty_cache()
gc.collect()

torch.Size([32, 91])


20

# Training the model
Implement your training process below. Report the test-accuracy after every epoch for the training run of the final model.

Hint: before training your model make sure to reset the seed in the training cell, as otherwise the seed may have changed due to previous training runs in the notebook

Note: If you implement automatic hyperparameter tuning, split the train set into train and validation subsets for the objective function.

In [8]:
### TRAINING FUNCTIONS ###

def train(model, dataloader, optimizer, loss_fn, updates):
    """
    One epoch of model training.
    """
    model.train()

    running_loss = 0.0
    running_accuracy = 0.0

    total_loss = 0.0
    total_accuracy = 0.0
    
    for idx, data in enumerate(dataloader):
        images, labels = data[0].to(device), data[1].to(device)

        # Forward propagate
        optimizer.zero_grad()
        outputs = model(images)
        correct = sum(labels == torch.argmax(outputs, 1)).item()
        acc = correct/batch_size
        running_accuracy += acc
        total_accuracy += acc

        # Back-propagate and optimize
        loss = loss_fn(outputs, labels)
        running_loss += loss.item()
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        # Print a snapshot of the training
        if idx % updates == (updates-1):
            avg_loss_across_batches = running_loss / updates
            avg_acc_across_batches = (running_accuracy / updates) * 100
            print('Batch {}, Loss: {:.3f}, Accuracy: {:.2f}%'.format(idx+1, avg_loss_across_batches, avg_acc_across_batches))
            running_loss = 0.0
            running_accuracy = 0.0

    avg_loss = total_loss / (idx+1)
    avg_accuracy = (total_accuracy / (idx+1)) * 100

    print('Training loss: {:.3f}, accuracy: {:.2f}%'.format(avg_loss, avg_accuracy))
    return avg_accuracy, avg_loss


def accuracy(model, dataloader, loss_fn):
    """
    One test accuracy test, can be used for validation as well.
    """
    model.eval()

    running_loss = 0.0
    running_accuracy = 0.0
    for data in iter(dataloader):
        images, labels = data[0].to(device), data[1].to(device)

        with torch.no_grad():
            outputs = model(images)
            correct = sum(labels == torch.argmax(outputs, 1)).item()
            running_accuracy += correct/batch_size
            
            loss = loss_fn(outputs, labels)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(dataloader)
    avg_acc_across_batches = (running_accuracy / len(dataloader)) * 100
    
    print('Testing loss: {:.3f}, accuracy: {:.2f}%'.format(avg_loss_across_batches, avg_acc_across_batches))
    return avg_acc_across_batches, avg_loss_across_batches


def run_epochs(num_epochs, model, optimizer, loss_fn, scheduler=None, epoch=0, best_acc=0.0, updates=500):
    """
    An entire training session.
    """
    upper = epoch+num_epochs
    train_accuracies, train_losses = [], []
    test_accuracies, test_losses = [], []
    
    for epoch in range(epoch, upper):
        # Start epoch
        print(f"Epoch {epoch+1} begin: {dt.now()}.")
        start = time()

        # Train the model
        train_acc, train_loss = train(model, train_loader, optimizer, loss_fn, updates)
        train_accuracies.append(train_acc)
        train_losses.append(train_loss)
        print('Completed training after {:.2f} seconds.'.format((time() - start)))
        check_point = time()

        # Test the model
        test_acc, test_loss = accuracy(model, test_loader, loss_fn)
        test_accuracies.append(test_acc)
        test_losses.append(test_loss)
        print('Completed testing after {:.2f} seconds.'.format((time() - check_point)))

        # Update learning rate
        if scheduler is not None:
            scheduler.step(test_loss)

        # Output an update
        print('Total epoch time: {:.2f} seconds.'.format((time() - start)))
        print('Epoch [{}/{}]'.format(epoch+1, upper))
        check_point = time()

        # Save best model
        if best_acc < test_acc:
            best_acc = test_acc
            print("Check-point!")
            torch.save(model.state_dict(), "./checkpoint.pth")
        print("####################################################################################################")
    return train_accuracies, train_losses, test_accuracies, test_losses

In [ ]:
### TRAINING ###

# Allow repeatability
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed)

# Create model object and connect it to the corresponding processing
model = ResNet([2, 3, 5, 2]).to(device)

# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, patience=4)

# Define mental state and start
train_accuracies, train_losses, test_accuracies, test_losses = run_epochs(num_epochs, model, optimizer, loss_fn, scheduler, updates=285)

Epoch 1 begin: 2025-05-16 16:46:15.933258.
Batch 285, Loss: 4.434, Accuracy: 3.06%
Batch 570, Loss: 4.287, Accuracy: 4.80%
Batch 855, Loss: 4.190, Accuracy: 6.40%
Batch 1140, Loss: 4.127, Accuracy: 6.96%
Batch 1425, Loss: 4.081, Accuracy: 7.98%
Completed training after 249.53 seconds.
Testing loss: 4.034, accuracy: 7.99%
Completed testing after 47.23 seconds.
Total epoch time: 296.77 seconds.
Epoch [1/50]
Check-point!
####################################################################################################
Epoch 2 begin: 2025-05-16 16:51:13.500746.
Batch 285, Loss: 3.999, Accuracy: 8.77%
Batch 570, Loss: 3.936, Accuracy: 10.00%
Batch 855, Loss: 3.896, Accuracy: 9.84%
Batch 1140, Loss: 3.854, Accuracy: 11.04%
Batch 1425, Loss: 3.822, Accuracy: 11.28%
Completed training after 250.95 seconds.
Testing loss: 4.066, accuracy: 9.65%
Completed testing after 48.13 seconds.
Total epoch time: 299.08 seconds.
Epoch [2/50]
Check-point!
####################################################

In [ ]:
print("Training")
print(train_accuracies)
print(train_losses)

print("Testing")
print(test_accuracies)
print(test_losses)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3))

# Plot the training and test accuracies to showcase the divergence at overfitting
ax1.plot(train_accuracies, label='Training')
ax1.plot(test_accuracies, label='Testing')
ax1.set_title('Accuracy')
ax1.legend()

# Plot the training and test losses to show the point of overfitting
ax2.plot(train_losses, label='Training')
ax2.plot(test_losses, label='Testing')
ax2.set_title('Loss')
ax2.legend()

plt.show()

In [48]:
random_seed, batch_size, lr, momentum, loss_fn, optimizer, scheduler

(ResNet(
   (layer1): Sequential(
     (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
     (1): SELU()
     (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   )
   (layer2): Sequential(
     (0): Block(
       (layer): Sequential(
         (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
         (1): SELU()
         (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
         (4): SELU()
         (5): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
         (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       )
       (p_layer): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
     )
     (1): Block(
       (layer): Sequential(
         (0): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
         (1): SELU()
         (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, aff

In [ ]:
model

In [9]:
del model
del loss_fn
del optimizer
del scheduler

torch.cuda.empty_cache()
gc.collect()

807

# Calculating model performance
Load the best version of your model ( which should be produced and saved by previous cells ), calculate and report the test accuracy.

In [51]:
### MODEL COMPARISONS ###

# Load the best model weights
model2 = ResNet([2, 3, 5, 2]).to(device)
model2.load_state_dict(torch.load("checkpoint.pth", weights_only=True))

# Calculate and present final scores
final_test_acc = accuracy(model2, test_loader)
print(f"Final Test Accuracy: {final_test_acc:.2f}%")


RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "layer2.0.layer.0.weight", "layer2.0.layer.0.bias", "layer2.0.layer.2.weight", "layer2.0.layer.2.bias", "layer2.0.layer.2.running_mean", "layer2.0.layer.2.running_var", "layer2.0.layer.3.weight", "layer2.0.layer.3.bias", "layer2.0.layer.5.weight", "layer2.0.layer.5.bias", "layer2.0.layer.6.weight", "layer2.0.layer.6.bias", "layer2.0.layer.6.running_mean", "layer2.0.layer.6.running_var", "layer2.0.p_layer.weight", "layer2.0.p_layer.bias", "layer2.1.layer.0.weight", "layer2.1.layer.0.bias", "layer2.1.layer.2.weight", "layer2.1.layer.2.bias", "layer2.1.layer.2.running_mean", "layer2.1.layer.2.running_var", "layer2.1.layer.3.weight", "layer2.1.layer.3.bias", "layer2.1.layer.5.weight", "layer2.1.layer.5.bias", "layer2.1.layer.6.weight", "layer2.1.layer.6.bias", "layer2.1.layer.6.running_mean", "layer2.1.layer.6.running_var", "layer2.2.layer.0.weight", "layer2.2.layer.0.bias", "layer2.2.layer.2.weight", "

# Summary of hyperparameters
Report the hyperparameters ( learning rate etc ) that you used in your final model for reproducibility.

# Simulation of random user
Pick 10 random pictures of the test set to simulate a user uploading images and report which categories occur how often in these: 1pt

In [ ]:
# Your code here
# Below an example showing the format of the code output

# Bonus point
Use an LLM (API) to generate a description of the food preference of a user based on 10 images that a potential user could provide. 
Please include an example of the output of your code, especially if you used an API other than the OpenAI API.

This should work well even with differing test images by setting different random seeds for the image selector.